## SPECT/CT Sensitivity

Calculates sensitivity as:

**True Positives / (True Positives + False Negatives)**

- True positives: Exact or Side matches  
- False negatives: Non-localizing scans

In [1]:
import pandas as pd 
preop_imaging_df = pd.read_excel('../../Data/final_imaging_data.xlsx')

In [18]:
def check_concordance(row, modality_col):
    imaging = row[modality_col]
    surgical = row['Surgical Findings']

    # Return Unknown if imaging or surgical data is missing
    if pd.isnull(imaging) or pd.isnull(surgical):
        return 'Unknown'

    # Handle special cases
    if imaging == 'No Scan':
        return 'No Scan'
    if imaging == 'Non-localizing':
        return 'Non-localizing'
    if 'hyperplasia' in surgical.lower():
        return 'Incorrect Localization'

    imaging_glands = [g.strip().lower() for g in imaging.split(',')]
    surgical_glands = [g.strip().lower() for g in surgical.split(',')]
    imaging_set = set(imaging_glands)
    surgical_set = set(surgical_glands)

    if surgical_set.issubset(imaging_set):
        return 'Yes (Exact)'
    if imaging_set & surgical_set:
        return 'Yes (Partial)'

    imaging_sides = ['left' for ig in imaging_glands if 'left' in ig] + \
                    ['right' for ig in imaging_glands if 'right' in ig]
    surgical_sides = ['left' for sg in surgical_glands if 'left' in sg] + \
                     ['right' for sg in surgical_glands if 'right' in sg]

    if any(side in surgical_sides for side in imaging_sides):
        if len(surgical_glands) > 1:
            return 'Yes (Partial)'
        else:
            return 'Yes (Side)'

    return 'Incorrect Localization'

    

In [19]:
# Define modality
modality = 'SPECT/CT'

# Calculate concordance using your robust function
preop_imaging_df['SPECT/CT Calculated Concordance'] = preop_imaging_df.apply(
    lambda row: check_concordance(row, modality),
    axis=1
)


localized_spect = preop_imaging_df[
    preop_imaging_df['SPECT/CT Calculated Concordance'].isin(['Yes (Exact)', 'Yes (Side)', 'Yes (Partial)'])
].copy()

nonlocalized_spect = preop_imaging_df[
    preop_imaging_df['SPECT/CT'] == 'Non-localizing'
].copy()


true_positives = localized_spect['SPECT/CT Calculated Concordance'].isin(['Yes (Exact)', 'Yes (Side)']).sum()
false_negatives = nonlocalized_spect.shape[0]
false_negatives 
true_positives

139

In [20]:
sensitivity = true_positives / (true_positives + false_negatives)
print(f"SPECT/CT Sensitivity: {sensitivity:.2%} ({true_positives} True Positive / {true_positives + false_negatives} total with disease)")

SPECT/CT Sensitivity: 72.77% (139 True Positive / 191 total with disease)
